In [1]:
import scipy.stats as sts
from scipy.stats import linregress
import geopandas as gpd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
os.getcwd()
from sqlalchemy import create_engine
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

# <center> Life Expectancy </center>

In [2]:
# Parth Korat
# Read in life expectancy dataset
life_expectancy_df = pd.read_csv("Resources\lifeExpectancyAtBirth.csv")

# columns in the dataset
life_expectancy_df.columns

Index(['Location', 'Period', 'Indicator', 'Dim1', 'First Tooltip'], dtype='object')

In [3]:
# Parth Korat
# Set the Period as the index
life_expectancy_df_period = life_expectancy_df[life_expectancy_df['Dim1'] == 'Both sexes'].set_index('Period')

# Get the number of periods in a given country
life_expectancy_df_year = life_expectancy_df_period['Location'].value_counts()
life_expectancy_df_year

Canada                5
Malta                 4
Congo                 4
Nepal                 4
Guinea                4
                     ..
Suriname              4
Germany               4
Zimbabwe              4
Sudan                 2
Sudan (until 2011)    2
Name: Location, Length: 184, dtype: int64

In [4]:
# Parth Korat
# Remove countries with less than 4 years of life expectancy data
life_expectancy_df_clean = life_expectancy_df_period[life_expectancy_df_period['Location'].isin(
                            life_expectancy_df_year[life_expectancy_df_year >= 4].index)]
life_expectancy_df_clean

,Location,Indicator,Dim1,First Tooltip
Period,,,,
2019,Afghanistan,Life expectancy at birth (years),Both sexes,63.21
2015,Afghanistan,Life expectancy at birth (years),Both sexes,61.65
2010,Afghanistan,Life expectancy at birth (years),Both sexes,59.94
2000,Afghanistan,Life expectancy at birth (years),Both sexes,54.99
2019,Albania,Life expectancy at birth (years),Both sexes,78.00
...,...,...,...,...
2000,Zambia,Life expectancy at birth (years),Both sexes,44.46
2019,Zimbabwe,Life expectancy at birth (years),Both sexes,60.68
2015,Zimbabwe,Life expectancy at birth (years),Both sexes,58.48


In [5]:
# Parth Korat
# Remove 1920 period from the data set
life_expectancy_df_clean = life_expectancy_df_clean.drop(1920)
life_expectancy_df_clean = life_expectancy_df_clean.drop(['Dim1', 'Indicator'], axis=1)

In [6]:
# Parth Korat
# Rename columns and reset the index
life_expectancy_df_clean.columns = ['Location', 'Life Expectancy in Years']
life_expectancy_df_clean = life_expectancy_df_clean.reset_index()
life_expectancy_df_clean

,Period,Location,Life Expectancy in Years
0,2019,Afghanistan,63.21
1,2015,Afghanistan,61.65
2,2010,Afghanistan,59.94
3,2000,Afghanistan,54.99
4,2019,Albania,78.00
...,...,...,...
723,2000,Zambia,44.46
724,2019,Zimbabwe,60.68
725,2015,Zimbabwe,58.48
726,2010,Zimbabwe,51.49


In [7]:
# Number of unique countries in life expectancy dataset
len(life_expectancy_df_clean['Location'].unique())

182

# <center> Handwashing </center>

In [8]:
# Parth Korat
# read basicHandWashing.csv
life_handwashing_df = pd.read_csv('Resources/Infrastructure/basicHandWashing.csv')
life_handwashing_df.head()

,Location,Indicator,Period,Dim1,First Tooltip
0,Afghanistan,Population with basic handwashing facilities a...,2017,Total,37.75
1,Afghanistan,Population with basic handwashing facilities a...,2017,Urban,63.58
2,Afghanistan,Population with basic handwashing facilities a...,2017,Rural,29.02
3,Afghanistan,Population with basic handwashing facilities a...,2016,Total,37.67
4,Afghanistan,Population with basic handwashing facilities a...,2016,Urban,63.58


In [9]:
# Parth Korat
# Drop Indicator and Dim1 column, and get Total for Dim1 only.
life_handwashing_df_clean = life_handwashing_df[life_handwashing_df['Dim1'] == 'Total'].drop(['Indicator', 'Dim1'], axis=1)
life_handwashing_df_clean['First Tooltip'].describe()

count    921.000000
mean      47.501911
std       32.774406
min        1.030000
25%       13.980000
50%       44.520000
75%       81.740000
max      100.000000
Name: First Tooltip, dtype: float64

In [10]:
# Parth Korat
# using replace to change the Period values
life_handwashing_df_clean = life_handwashing_df_clean.replace(
    [2017, 2016, 2015, 2014, 2013, 2012, 2011, 2010, 2009, 2008, 2007, 2006, 2005, 2004, 2003, 2002, 2001, 2000],
    [2019, 2019, 2015, 2015, 2015, 2015, 2015, 2010, 2010, 2010, 2010, 2010, 2005, 2005, 2005, 2005, 2005, 2000]
)

life_handwashing_df_clean.columns = ['Location', 'Period', 'Handwashing Facilities at Home (%)']
life_handwashing_df_clean

,Location,Period,Handwashing Facilities at Home (%)
0,Afghanistan,2019,37.75
3,Afghanistan,2019,37.67
6,Afghanistan,2015,37.59
9,Afghanistan,2015,37.52
12,Afghanistan,2015,37.44
...,...,...,...
2711,Zimbabwe,2015,36.82
2714,Zimbabwe,2015,36.84
2717,Zimbabwe,2015,36.86
2720,Zimbabwe,2015,36.89


In [11]:
life_handwashing_df_clean = life_handwashing_df_clean.groupby(['Location', 'Period']).mean().reset_index()
life_handwashing_df_clean

,Location,Period,Handwashing Facilities at Home (%)
0,Afghanistan,2005,36.870
1,Afghanistan,2010,37.080
2,Afghanistan,2015,37.444
3,Afghanistan,2019,37.710
4,Algeria,2010,83.020
...,...,...,...
271,Zambia,2010,13.200
272,Zambia,2015,13.508
273,Zambia,2019,13.885
274,Zimbabwe,2015,36.868


In [12]:
# Number of unique countries in handwashing dataset
len(life_handwashing_df_clean['Location'].unique())

96

# <center> Drinking Water </center>

In [13]:
# Parth Korat
# read basicDrinkingWaterServices.csv
life_drinkingwater_df = pd.read_csv('Resources/Infrastructure/basicDrinkingWaterServices.csv')
life_drinkingwater_df.head()

,Location,Period,Indicator,First Tooltip
0,Afghanistan,2017,Population using at least basic drinking-water...,57.32
1,Afghanistan,2016,Population using at least basic drinking-water...,54.84
2,Afghanistan,2015,Population using at least basic drinking-water...,52.39
3,Afghanistan,2014,Population using at least basic drinking-water...,49.96
4,Afghanistan,2013,Population using at least basic drinking-water...,47.56


In [14]:
# Parth Korat
# Drop Indicator column
life_drinkingwater_df_clean = life_drinkingwater_df.drop('Indicator', axis=1)
life_drinkingwater_df_clean['First Tooltip'].describe()

count    3455.000000
mean       77.847103
std        23.987570
min        10.130000
25%        59.895000
50%        87.650000
75%        98.460000
max       100.000000
Name: First Tooltip, dtype: float64

In [15]:
# Parth Korat
# using replace to change the Period values
life_drinkingwater_df_clean = life_drinkingwater_df_clean.replace(
    [2017, 2016, 2015, 2014, 2013, 2012, 2011, 2010, 2009, 2008, 2007, 2006, 2005, 2004, 2003, 2002, 2001, 2000],
    [2019, 2019, 2015, 2015, 2015, 2015, 2015, 2010, 2010, 2010, 2010, 2010, 2005, 2005, 2005, 2005, 2005, 2000]
)

life_drinkingwater_df_clean.columns = ['Location', 'Period', 'Drinking Water Facilities at Home (%)']
life_drinkingwater_df_clean

,Location,Period,Drinking Water Facilities at Home (%)
0,Afghanistan,2019,57.32
1,Afghanistan,2019,54.84
2,Afghanistan,2015,52.39
3,Afghanistan,2015,49.96
4,Afghanistan,2015,47.56
...,...,...,...
3450,Zimbabwe,2005,57.94
3451,Zimbabwe,2005,58.59
3452,Zimbabwe,2005,59.23
3453,Zimbabwe,2005,59.88


In [16]:
life_drinkingwater_df_clean = life_drinkingwater_df_clean.groupby(['Location', 'Period']).mean().reset_index()
life_drinkingwater_df_clean

,Location,Period,Drinking Water Facilities at Home (%)
0,Afghanistan,2000,21.620
1,Afghanistan,2005,25.580
2,Afghanistan,2010,35.994
3,Afghanistan,2015,47.588
4,Afghanistan,2019,56.080
...,...,...,...
952,Zimbabwe,2000,59.880
953,Zimbabwe,2005,58.588
954,Zimbabwe,2010,55.402
955,Zimbabwe,2015,52.270


In [17]:
# Number of unique countries in drinking water dataset
len(life_drinkingwater_df_clean['Location'].unique())

195

# <center> Birth Attended (%) </center>

In [18]:
birth_skilledprof_df = pd.read_csv("Resources/Healthcare/birthAttendedBySkilledPersonal.csv")
birth_skilledprof_df

,Location,Period,Indicator,First Tooltip
0,Afghanistan,2018,Births attended by skilled health personnel (%),58.8
1,Afghanistan,2017,Births attended by skilled health personnel (%),53.4
2,Afghanistan,2015,Births attended by skilled health personnel (%),50.5
3,Afghanistan,2014,Births attended by skilled health personnel (%),45.2
4,Afghanistan,2012,Births attended by skilled health personnel (%),39.9
...,...,...,...,...
1750,Zimbabwe,2015,Births attended by skilled health personnel (%),78.1
1751,Zimbabwe,2014,Births attended by skilled health personnel (%),80.0
1752,Zimbabwe,2011,Births attended by skilled health personnel (%),66.2
1753,Zimbabwe,2009,Births attended by skilled health personnel (%),60.2


In [19]:
birth_skilledprof_df_clean = birth_skilledprof_df.drop('Indicator', axis=1)
birth_skilledprof_df_clean['First Tooltip'].describe()

count    1755.000000
mean       92.045442
std        16.557089
min         5.700000
25%        95.350000
50%        99.000000
75%        99.800000
max       100.000000
Name: First Tooltip, dtype: float64

In [20]:
birth_skilledprof_df_clean = birth_skilledprof_df_clean.replace(
    [2019, 2018, 2017, 2016, 2015, 2014, 2013, 2012, 2011, 2010, 2009, 2008, 2007, 2006, 2005, 2004, 2003, 2002, 2001, 2000],
    [2019, 2019, 2019, 2019, 2015, 2015, 2015, 2015, 2015, 2010, 2010, 2010, 2010, 2010, 2005, 2005, 2005, 2005, 2005, 2000]
)

birth_skilledprof_df_clean.columns = ['Location', 'Period', 'Births Attended by Skilled Professional(%)']
birth_skilledprof_df_clean

,Location,Period,Births Attended by Skilled Professional(%)
0,Afghanistan,2019,58.8
1,Afghanistan,2019,53.4
2,Afghanistan,2015,50.5
3,Afghanistan,2015,45.2
4,Afghanistan,2015,39.9
...,...,...,...
1750,Zimbabwe,2015,78.1
1751,Zimbabwe,2015,80.0
1752,Zimbabwe,2015,66.2
1753,Zimbabwe,2010,60.2


In [21]:
birth_skilledprof_df_clean = birth_skilledprof_df_clean.groupby(['Location', 'Period']).mean().reset_index()
birth_skilledprof_df_clean

,Location,Period,Births Attended by Skilled Professional(%)
0,Afghanistan,2005,14.300000
1,Afghanistan,2010,34.300000
2,Afghanistan,2015,43.550000
3,Afghanistan,2019,56.100000
4,Albania,2005,99.800000
...,...,...,...
657,Zambia,2010,45.200000
658,Zambia,2015,63.300000
659,Zimbabwe,2010,64.350000
660,Zimbabwe,2015,74.766667


In [22]:
# Number of unique countries in birth attended by skilled professional dataset
len(birth_skilledprof_df_clean['Location'].unique())

186

# <center> Available Doctors </center>

In [23]:
medical_doctors_df = pd.read_csv("Resources/Healthcare/medicalDoctors.csv")
medical_doctors_df

,Location,Period,Indicator,First Tooltip
0,Afghanistan,2016,"Medical doctors (per 10,000)",2.78
1,Afghanistan,2015,"Medical doctors (per 10,000)",2.85
2,Afghanistan,2014,"Medical doctors (per 10,000)",2.98
3,Afghanistan,2013,"Medical doctors (per 10,000)",2.85
4,Afghanistan,2012,"Medical doctors (per 10,000)",2.41
...,...,...,...,...
2501,Zimbabwe,2005,"Medical doctors (per 10,000)",1.26
2502,Zimbabwe,2004,"Medical doctors (per 10,000)",1.74
2503,Zimbabwe,2000,"Medical doctors (per 10,000)",1.26
2504,Zimbabwe,1995,"Medical doctors (per 10,000)",1.43


In [24]:
medical_doctors_df_clean = medical_doctors_df.drop('Indicator', axis=1)
medical_doctors_df_clean['First Tooltip'].describe()

count    2506.000000
mean       20.685012
std        14.299267
min         0.130000
25%         7.782500
50%        21.280000
75%        31.660000
max        84.220000
Name: First Tooltip, dtype: float64

In [25]:
medical_doctors_df_clean = medical_doctors_df_clean.loc[medical_doctors_df_clean['Period'] > 1995]
medical_doctors_df_clean

,Location,Period,First Tooltip
0,Afghanistan,2016,2.78
1,Afghanistan,2015,2.85
2,Afghanistan,2014,2.98
3,Afghanistan,2013,2.85
4,Afghanistan,2012,2.41
...,...,...,...
2499,Zimbabwe,2008,0.60
2500,Zimbabwe,2007,0.54
2501,Zimbabwe,2005,1.26
2502,Zimbabwe,2004,1.74


In [26]:
medical_doctors_df_clean = medical_doctors_df_clean.replace(
    [2018, 2017, 2016, 2015, 2014, 2013, 2012, 2011, 2010, 2009, 2008, 2007, 2006, 2005, 2004, 2003, 2002, 2001, 2000, 1999, 1998, 1997, 1996],
    [2019, 2019, 2019, 2015, 2015, 2015, 2015, 2015, 2010, 2010, 2010, 2010, 2010, 2005, 2005, 2005, 2005, 2005, 2000, 2000, 2000, 2000, 2000]
)

medical_doctors_df_clean.columns = ['Location', 'Period', 'Medical Doctors Available (%)']
medical_doctors_df_clean

,Location,Period,Medical Doctors Available (%)
0,Afghanistan,2019,2.78
1,Afghanistan,2015,2.85
2,Afghanistan,2015,2.98
3,Afghanistan,2015,2.85
4,Afghanistan,2015,2.41
...,...,...,...
2499,Zimbabwe,2010,0.60
2500,Zimbabwe,2010,0.54
2501,Zimbabwe,2005,1.26
2502,Zimbabwe,2005,1.74


In [27]:
medical_doctors_df_clean = medical_doctors_df_clean.groupby(['Location', 'Period']).mean().reset_index()
medical_doctors_df_clean

,Location,Period,Medical Doctors Available (%)
0,Afghanistan,2005,1.900000
1,Afghanistan,2010,1.916000
2,Afghanistan,2015,2.722000
3,Afghanistan,2019,2.780000
4,Albania,2000,13.855000
...,...,...,...
765,Zimbabwe,2000,1.260000
766,Zimbabwe,2005,1.500000
767,Zimbabwe,2010,0.905000
768,Zimbabwe,2015,1.104000


In [28]:
# Number of unique countries in medical doctors avaliable dataset
len(medical_doctors_df_clean['Location'].unique())

194

# <center> Avaliable Nurses or Midwives </center>

In [29]:
nurse_midwife_df = pd.read_csv("Resources/Healthcare/nursingAndMidwife.csv")
nurse_midwife_df

,Location,Period,Indicator,First Tooltip
0,Afghanistan,2017,"Nursing and midwifery personnel (per 10,000)",1.76
1,Afghanistan,2016,"Nursing and midwifery personnel (per 10,000)",1.48
2,Afghanistan,2015,"Nursing and midwifery personnel (per 10,000)",1.30
3,Afghanistan,2014,"Nursing and midwifery personnel (per 10,000)",1.48
4,Afghanistan,2013,"Nursing and midwifery personnel (per 10,000)",2.50
...,...,...,...,...
2582,Zimbabwe,2011,"Nursing and midwifery personnel (per 10,000)",13.20
2583,Zimbabwe,2010,"Nursing and midwifery personnel (per 10,000)",8.29
2584,Zimbabwe,2009,"Nursing and midwifery personnel (per 10,000)",9.49
2585,Zimbabwe,1995,"Nursing and midwifery personnel (per 10,000)",15.27


In [30]:
nurse_midwife_df_clean = nurse_midwife_df.drop('Indicator', axis=1)
nurse_midwife_df_clean['First Tooltip'].describe()

count    2587.000000
mean       45.417905
std        38.810124
min         0.012000
25%        11.835000
50%        37.450000
75%        66.165000
max       201.600000
Name: First Tooltip, dtype: float64

In [31]:
nurse_midwife_df_clean = nurse_midwife_df_clean.loc[nurse_midwife_df_clean['Period'] > 1995]

In [32]:
nurse_midwife_df_clean = nurse_midwife_df_clean.replace(
    [2018, 2017, 2016, 2015, 2014, 2013, 2012, 2011, 2010, 2009, 2008, 2007, 2006, 2005, 2004, 2003, 2002, 2001, 2000, 1999, 1998, 1997, 1996],
    [2019, 2019, 2019, 2015, 2015, 2015, 2015, 2015, 2010, 2010, 2010, 2010, 2010, 2005, 2005, 2005, 2005, 2005, 2000, 2000, 2000, 2000, 2000]
)

nurse_midwife_df_clean.columns = ['Location', 'Period', 'Avaliable Nurses and Midwives (%)']
nurse_midwife_df_clean

,Location,Period,Avaliable Nurses and Midwives (%)
0,Afghanistan,2019,1.76
1,Afghanistan,2019,1.48
2,Afghanistan,2015,1.30
3,Afghanistan,2015,1.48
4,Afghanistan,2015,2.50
...,...,...,...
2580,Zimbabwe,2015,14.02
2581,Zimbabwe,2015,11.85
2582,Zimbabwe,2015,13.20
2583,Zimbabwe,2010,8.29


In [33]:
nurse_midwife_df_clean = nurse_midwife_df_clean.groupby(['Location', 'Period']).mean().reset_index()
nurse_midwife_df_clean

,Location,Period,Avaliable Nurses and Midwives (%)
0,Afghanistan,2005,5.820000
1,Afghanistan,2010,5.102500
2,Afghanistan,2015,1.760000
3,Afghanistan,2019,1.620000
4,Albania,2000,42.808000
...,...,...,...
801,Zambia,2015,1.860000
802,Zambia,2019,9.470000
803,Zimbabwe,2010,8.890000
804,Zimbabwe,2015,13.654000


In [34]:
# Number of unique countries in avaliable nurses and midwives dataset
len(nurse_midwife_df_clean['Location'].unique())

194

# <center> Student/Teacher Ratio </center>

In [36]:
teaching_staff_df = pd.read_csv("Resources\Education\Teaching_Staff_in_Education.csv", encoding = "ISO-8859-1")
teaching_staff_df

,Region/Country/Area,Unnamed: 1,Year,Series,Value,Footnotes,Source
0,1,Total,NaN,NaN,NaN,NaN,NaN
1,1,Total,NaN,NaN,NaN,NaN,NaN
2,1,Total,NaN,NaN,NaN,NaN,NaN
3,1,Total,NaN,NaN,NaN,NaN,NaN
4,1,Total,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
4539,716,Zimbabwe,2013.0,Pupil-teacher ratio in secondary education,22.4835,NaN,United Nations Educational
4540,716,Zimbabwe,2013.0,Teachers at tertiary level (thousands),5.5370,NaN,United Nations Educational
4541,716,Zimbabwe,2013.0,Pupil-teacher ratio in tertiary education,16.9975,NaN,United Nations Educational
4542,716,Zimbabwe,2015.0,Teachers at tertiary level (thousands),7.5610,NaN,United Nations Educational


In [37]:
#only want data for pupil-teacher ratio in primary education
pupil_teacher_ratio_primary_df = teaching_staff_df.loc[(teaching_staff_df['Series'] == 'Pupil-teacher ratio in primary education')]
pupil_teacher_ratio_primary_df = pupil_teacher_ratio_primary_df.drop(['Region/Country/Area', 'Series', 'Footnotes', 'Source'], axis=1)
pupil_teacher_ratio_primary_df

,Unnamed: 1,Year,Value
21,Northern Africa,2005.0,24.3807
26,Northern Africa,2010.0,24.2881
31,Northern Africa,2015.0,21.9184
36,Northern Africa,2018.0,22.7634
41,Sub-Saharan Africa,2005.0,44.0129
...,...,...,...
4525,Zambia,2010.0,52.9851
4527,Zambia,2015.0,42.7418
4529,Zambia,2017.0,42.0600
4531,Zimbabwe,2003.0,38.5559


In [38]:
pupil_teacher_ratio_primary_df = pupil_teacher_ratio_primary_df.replace(
    [2018, 2017, 2016, 2015, 2014, 2013, 2012, 2011, 2010, 2009, 2008, 2007, 2006, 2005, 2004, 2003, 2002, 2001, 2000],
    [2019, 2019, 2019, 2015, 2015, 2015, 2015, 2015, 2010, 2010, 2010, 2010, 2010, 2005, 2005, 2005, 2005, 2005, 2000]
)

pupil_teacher_ratio_primary_df.columns = ['Location', 'Period', 'Student Teacher Ratio']
pupil_teacher_ratio_primary_df

,Location,Period,Student Teacher Ratio
21,Northern Africa,2005.0,24.3807
26,Northern Africa,2010.0,24.2881
31,Northern Africa,2015.0,21.9184
36,Northern Africa,2019.0,22.7634
41,Sub-Saharan Africa,2005.0,44.0129
...,...,...,...
4525,Zambia,2010.0,52.9851
4527,Zambia,2015.0,42.7418
4529,Zambia,2019.0,42.0600
4531,Zimbabwe,2005.0,38.5559


In [39]:
pupil_teacher_ratio_primary_df = pupil_teacher_ratio_primary_df.groupby(['Location', 'Period']).mean().reset_index()
pupil_teacher_ratio_primary_df.dtypes

Location                  object
Period                   float64
Student Teacher Ratio    float64
dtype: object

In [40]:
# Number of unique countries in avaliable student/teacher ratio dataset
len(pupil_teacher_ratio_primary_df['Location'].unique())

214

# <center> GDP Spent on Education (%) </center>

In [41]:
public_expenditure_df = pd.read_csv("Resources\Education\Public_Expenditure_on_Education.csv", encoding = "ISO-8859-1")
public_expenditure_df

,ï»¿T08,Public expenditure on education (new),Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,Region/Country/Area,NaN,Year,Series,Value,Footnotes
1,4,Afghanistan,2010,Current expenditure other than staff compensat...,9.3032,NaN
2,4,Afghanistan,2010,All staff compensation as % of total expenditu...,72.3783,NaN
3,4,Afghanistan,2010,Capital expenditure as % of total expenditure ...,18.3186,NaN
4,4,Afghanistan,2010,Expenditure by level of education: primary (as...,62.1365,NaN
...,...,...,...,...,...,...
4976,716,Zimbabwe,2014,Expenditure by level of education: tertiary (a...,16.8321,NaN
4977,716,Zimbabwe,2014,Government expenditure on education (% of gove...,30.0152,NaN
4978,716,Zimbabwe,2014,Public expenditure on education (% of GDP),6.1384,NaN
4979,716,Zimbabwe,2018,Government expenditure on education (% of gove...,19.0398,Estimate.


In [42]:
#only want data for public expenditures as a % of GDP
education_spending = public_expenditure_df.loc[(public_expenditure_df['Unnamed: 3'] == 'Public expenditure on education (% of GDP)')]
education_spending = education_spending.drop(['ï»¿T08', 'Unnamed: 3', 'Unnamed: 5'], axis=1)
education_spending

,Public expenditure on education (new),Unnamed: 2,Unnamed: 4
8,Afghanistan,2010,3.4795
16,Afghanistan,2015,3.2558
20,Afghanistan,2017,4.0589
22,Albania,2005,3.2816
24,Albania,2007,3.2759
...,...,...,...
4965,Zambia,2017,3.7297
4967,Zambia,2018,4.6674
4972,Zimbabwe,2010,1.5441
4978,Zimbabwe,2014,6.1384


In [43]:
education_spending.columns = ['Location', 'Period', 'GDP spent on Education (%)']
education_spending = education_spending.astype({'Period': 'float64', 'GDP spent on Education (%)': 'float64'})
education_spending

,Location,Period,GDP spent on Education (%)
8,Afghanistan,2010.0,3.4795
16,Afghanistan,2015.0,3.2558
20,Afghanistan,2017.0,4.0589
22,Albania,2005.0,3.2816
24,Albania,2007.0,3.2759
...,...,...,...
4965,Zambia,2017.0,3.7297
4967,Zambia,2018.0,4.6674
4972,Zimbabwe,2010.0,1.5441
4978,Zimbabwe,2014.0,6.1384


In [44]:
education_spending = education_spending.replace(
    [2019, 2018, 2017, 2016, 2015, 2014, 2013, 2012, 2011, 2010, 2009, 2008, 2007, 2006, 2005, 2004, 2003, 2002, 2001, 2000],
    [2019, 2019, 2019, 2019, 2015, 2015, 2015, 2015, 2015, 2010, 2010, 2010, 2010, 2010, 2005, 2005, 2005, 2005, 2005, 2000]
)

education_spending

,Location,Period,GDP spent on Education (%)
8,Afghanistan,2010.0,3.4795
16,Afghanistan,2015.0,3.2558
20,Afghanistan,2019.0,4.0589
22,Albania,2005.0,3.2816
24,Albania,2010.0,3.2759
...,...,...,...
4965,Zambia,2019.0,3.7297
4967,Zambia,2019.0,4.6674
4972,Zimbabwe,2010.0,1.5441
4978,Zimbabwe,2015.0,6.1384


In [45]:
education_spending = education_spending.groupby(['Location', 'Period']).mean().reset_index()
education_spending

,Location,Period,GDP spent on Education (%)
0,Afghanistan,2010.0,3.479500
1,Afghanistan,2015.0,3.255800
2,Afghanistan,2019.0,4.058900
3,Albania,2005.0,3.281600
4,Albania,2010.0,3.275900
...,...,...,...
612,Zambia,2015.0,4.624300
613,Zambia,2019.0,4.048333
614,Zimbabwe,2010.0,1.544100
615,Zimbabwe,2015.0,6.138400


In [46]:
# Number of unique countries in % gdp spend on education dataset
len(education_spending['Location'].unique())

192

# <center> Literacy Rates </center>

In [47]:
literacy_rates_df = pd.read_csv("Resources\Education\cross-country-literacy-rates.csv")
literacy_rates_df

,Entity,Code,Year,"Literacy rates (World Bank, CIA World Factbook, and other sources)",Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,Afghanistan,AFG,1979,18.157681,NaN,NaN,NaN,NaN
1,Afghanistan,AFG,2011,31.741119,NaN,NaN,NaN,NaN
2,Afghanistan,AFG,2015,38.168041,NaN,NaN,NaN,NaN
3,Albania,ALB,2001,98.712982,NaN,NaN,NaN,NaN
4,Albania,ALB,2008,95.938637,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
1418,Zambia,ZMB,2015,85.117264,NaN,NaN,NaN,NaN
1419,Zimbabwe,ZWE,1982,77.794167,NaN,NaN,NaN,NaN
1420,Zimbabwe,ZWE,1992,83.512581,NaN,NaN,NaN,NaN
1421,Zimbabwe,ZWE,2011,83.582710,NaN,NaN,NaN,NaN


In [48]:
literacy_rates_df = literacy_rates_df.loc[literacy_rates_df['Year'] > 1995]
literacy_rates_df

,Entity,Code,Year,"Literacy rates (World Bank, CIA World Factbook, and other sources)",Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
1,Afghanistan,AFG,2011,31.741119,NaN,NaN,NaN,NaN
2,Afghanistan,AFG,2015,38.168041,NaN,NaN,NaN,NaN
3,Albania,ALB,2001,98.712982,NaN,NaN,NaN,NaN
4,Albania,ALB,2008,95.938637,NaN,NaN,NaN,NaN
5,Albania,ALB,2011,96.845299,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
1416,Zambia,ZMB,2007,61.428291,NaN,NaN,NaN,NaN
1417,Zambia,ZMB,2010,83.007668,NaN,NaN,NaN,NaN
1418,Zambia,ZMB,2015,85.117264,NaN,NaN,NaN,NaN
1421,Zimbabwe,ZWE,2011,83.582710,NaN,NaN,NaN,NaN


In [49]:
literacy_rates_df = literacy_rates_df.drop(['Code', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7'], axis=1)
literacy_rates_df

,Entity,Year,"Literacy rates (World Bank, CIA World Factbook, and other sources)"
1,Afghanistan,2011,31.741119
2,Afghanistan,2015,38.168041
3,Albania,2001,98.712982
4,Albania,2008,95.938637
5,Albania,2011,96.845299
...,...,...,...
1416,Zambia,2007,61.428291
1417,Zambia,2010,83.007668
1418,Zambia,2015,85.117264
1421,Zimbabwe,2011,83.582710


In [50]:
literacy_rates_df = literacy_rates_df.replace(
    [2015, 2014, 2013, 2012, 2011, 2010, 2009, 2008, 2007, 2006, 2005, 2004, 2003, 2002, 2001, 2000, 1999, 1998, 1997, 1996],
    [2015, 2015, 2015, 2015, 2015, 2010, 2010, 2010, 2010, 2010, 2005, 2005, 2005, 2005, 2005, 2000, 2000, 2000, 2000, 2000]
)

literacy_rates_df.columns = ['Location', 'Period', 'Literacy Rates']

In [51]:
literacy_rates_df = literacy_rates_df.groupby(['Location', 'Period']).mean().reset_index()
literacy_rates_df

,Location,Period,Literacy Rates
0,Afghanistan,2015,34.954580
1,Albania,2005,98.712982
2,Albania,2010,95.938637
3,Albania,2015,97.215391
4,Algeria,2005,69.873497
...,...,...,...
529,Zambia,2000,68.001793
530,Zambia,2005,69.149223
531,Zambia,2010,72.217980
532,Zambia,2015,85.117264


In [52]:
# Number of unique countries in literacy rates dataset
len(literacy_rates_df['Location'].unique())

244

# <center> Merge all data </center>

In [53]:
life_features_list = [life_expectancy_df_clean.set_index(['Location', 'Period']), life_drinkingwater_df_clean.set_index(['Location', 'Period']), life_handwashing_df_clean.set_index(['Location', 'Period']),
                        birth_skilledprof_df_clean.set_index(['Location', 'Period']), education_spending.set_index(['Location', 'Period']), literacy_rates_df.set_index(['Location', 'Period']),
                        medical_doctors_df_clean.set_index(['Location', 'Period']), nurse_midwife_df_clean.set_index(['Location', 'Period']), pupil_teacher_ratio_primary_df.set_index(['Location', 'Period'])
]

In [54]:
life_features_df = pd.concat(life_features_list, axis=1, join='outer').reset_index()

life_features_df

,Location,Period,Life Expectancy in Years,Drinking Water Facilities at Home (%),Handwashing Facilities at Home (%),Births Attended by Skilled Professional(%),GDP spent on Education (%),Literacy Rates,Medical Doctors Available (%),Avaliable Nurses and Midwives (%),Student Teacher Ratio
0,Afghanistan,2000.0,54.99,21.620,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,2005.0,NaN,25.580,36.870,14.300000,NaN,NaN,1.900000,5.820000,NaN
2,Afghanistan,2010.0,59.94,35.994,37.080,34.300000,3.4795,NaN,1.916000,5.102500,43.341267
3,Afghanistan,2015.0,61.65,47.588,37.444,43.550000,3.2558,34.954580,2.722000,1.760000,43.388400
4,Afghanistan,2019.0,63.21,56.080,37.710,56.100000,4.0589,NaN,2.780000,1.620000,48.789800
...,...,...,...,...,...,...,...,...,...,...,...
1248,Zimbabwe,2000.0,46.57,59.880,NaN,NaN,NaN,NaN,1.260000,NaN,NaN
1249,Zimbabwe,2005.0,NaN,58.588,NaN,NaN,NaN,NaN,1.500000,NaN,38.555900
1250,Zimbabwe,2010.0,51.49,55.402,NaN,64.350000,1.5441,NaN,0.905000,8.890000,NaN
1251,Zimbabwe,2015.0,58.48,52.270,36.868,74.766667,6.1384,85.228096,1.104000,13.654000,36.408200


In [75]:
life_features_df_clean = life_features_df[life_features_df['Life Expectancy in Years'].notna()]
life_features_df_clean

,Location,Period,Life Expectancy in Years,Drinking Water Facilities at Home (%),Handwashing Facilities at Home (%),Births Attended by Skilled Professional(%),GDP spent on Education (%),Literacy Rates,Medical Doctors Available (%),Avaliable Nurses and Midwives (%),Student Teacher Ratio
0,Afghanistan,2000.0,54.99,21.620,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,2010.0,59.94,35.994,37.080,34.300000,3.479500,NaN,1.916000,5.102500,43.341267
3,Afghanistan,2015.0,61.65,47.588,37.444,43.550000,3.255800,34.954580,2.722000,1.760000,43.388400
4,Afghanistan,2019.0,63.21,56.080,37.710,56.100000,4.058900,NaN,2.780000,1.620000,48.789800
5,Albania,2000.0,73.55,81.780,NaN,NaN,NaN,NaN,13.855000,42.808000,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1247,Zambia,2019.0,62.45,41.565,13.885,NaN,4.048333,NaN,6.750000,9.470000,42.060000
1248,Zimbabwe,2000.0,46.57,59.880,NaN,NaN,NaN,NaN,1.260000,NaN,NaN
1250,Zimbabwe,2010.0,51.49,55.402,NaN,64.350000,1.544100,NaN,0.905000,8.890000,NaN
1251,Zimbabwe,2015.0,58.48,52.270,36.868,74.766667,6.138400,85.228096,1.104000,13.654000,36.408200


                                                                        Rename the Columns in Dataframe

In [76]:
life_expectancy_df = life_features_df_clean.rename(columns={'Location':'country', 'Period':'year', 'Life Expectancy in Years':'expectancy', 'Drinking Water Facilities at Home (%)':'drinkingwater', 
                        'Handwashing Facilities at Home (%)':'handwash', 'Births Attended by Skilled Professional(%)':'birthattended','GDP spent on Education (%)':'gdpspent',
                        'Literacy Rates':'literacy', 'Medical Doctors Available (%)':'meddoctor', 'Avaliable Nurses and Midwives (%)':'nursemidwives', 
                        'Student Teacher Ratio':'stuteacherratio'})
life_expectancy_df.head()

,country,year,expectancy,drinkingwater,handwash,birthattended,gdpspent,literacy,meddoctor,nursemidwives,stuteacherratio
0,Afghanistan,2000.0,54.99,21.620,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,2010.0,59.94,35.994,37.080,34.30,3.4795,NaN,1.916,5.1025,43.341267
3,Afghanistan,2015.0,61.65,47.588,37.444,43.55,3.2558,34.95458,2.722,1.7600,43.388400
4,Afghanistan,2019.0,63.21,56.080,37.710,56.10,4.0589,NaN,2.780,1.6200,48.789800
5,Albania,2000.0,73.55,81.780,NaN,NaN,NaN,NaN,13.855,42.8080,NaN


Python code to Drop and Create the PostgreSQL table "life_expectancy" in the Database "ETLAssignment_dB"

In [77]:
# Start a PostgreSQL database session

psqlCon         = psycopg2.connect("dbname=ETLAssignment_dB user=postgres password=postgress");
psqlCon.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT);

# Open a database cursor

psqlCursor      = psqlCon.cursor();

Execute the below code only if the table exist in the database

In [78]:
# Name of the table to be deleted

tableName       = "life_expectancy";

# Form the SQL statement - DROP TABLE

dropTableStmt   = "DROP TABLE %s;"%tableName;

# Execute the drop table command

psqlCursor.execute(dropTableStmt);

Execute the following code if the table needs to be created

In [79]:
#Creating table as per requirement

sql = '''CREATE TABLE life_expectancy (
    id              SERIAL  PRIMARY KEY,
    country         VARCHAR NOT NULL ,
    year            INT     NOT NULL, 
    expectancy      decimal   NOT NULL,
    drinkingwater   decimal   NULL,
    handwash        decimal   NULL,
	birthattended   decimal   NULL,
	gdpspent        decimal   NULL,
    literacy        decimal   NULL,
	meddoctor       decimal   NULL,
	nursemidwives   decimal   NULL,
    stuteacherratio decimal   NULL
)'''

psqlCursor.execute(sql)
print("Table created successfully........")

Table created successfully........


In [80]:
#Closing the connection

# Free the resources

psqlCursor.close();

psqlCon.close();

Creating connection with the PostgreSQL to insert the Data from "life_features_df" Pandas Dataframe

In [81]:
rds_connection_string = "postgres:postgress@localhost:5432/ETLAssignment_dB"
engine = create_engine(f'postgresql://{rds_connection_string}')


In [71]:
engine.table_names()

['life_expectancy']

Inserting the data from Life expectancy Pandas dataframe to the "life_expectancy" PostgreSQL table

In [82]:
life_expectancy_df.to_sql(name='life_expectancy', con=engine, if_exists='append', index=False)

Select query to retrieve data from the PostgreSQL table

In [84]:
pd.read_sql_query('select * from life_expectancy', con=engine)

,id,country,year,expectancy,drinkingwater,handwash,birthattended,gdpspent,literacy,meddoctor,nursemidwives,stuteacherratio
0,1,Afghanistan,2000,54.99,21.620,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Afghanistan,2010,59.94,35.994,37.080,34.300000,3.479500,NaN,1.916000,5.102500,43.341267
2,3,Afghanistan,2015,61.65,47.588,37.444,43.550000,3.255800,34.954580,2.722000,1.760000,43.388400
3,4,Afghanistan,2019,63.21,56.080,37.710,56.100000,4.058900,NaN,2.780000,1.620000,48.789800
4,5,Albania,2000,73.55,81.780,NaN,NaN,NaN,NaN,13.855000,42.808000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
723,724,Zambia,2019,62.45,41.565,13.885,NaN,4.048333,NaN,6.750000,9.470000,42.060000
724,725,Zimbabwe,2000,46.57,59.880,NaN,NaN,NaN,NaN,1.260000,NaN,NaN
725,726,Zimbabwe,2010,51.49,55.402,NaN,64.350000,1.544100,NaN,0.905000,8.890000,NaN
726,727,Zimbabwe,2015,58.48,52.270,36.868,74.766667,6.138400,85.228096,1.104000,13.654000,36.408200
